<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>


# Notebook for Applied Data Science Capstone - Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto by Emil Fuerstenberg Haegg

## Part 1

#### Import and install libraries

In [1036]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
            
# Set diplay options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

#!pip install bs4  # uncomment if not already installed
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


### Scrape data from HTML tables into a DataFrame using BeautifulSoup and Pandas

In [1037]:
# Webpage URL
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1012118802"

In [1038]:
# Getting content in text format and store in variable 'data'
data  = requests.get(url).text

In [1039]:
# Creating beautifulsoup object on 'data'
soup = BeautifulSoup(data,"html.parser")

In [1040]:
# Find all html-tables in the webpage
tables = soup.find_all('table') # in html table is represented by the tag <table>

In [1041]:
# Check how many tables were found
len(tables)

3

In [1042]:
# Indices for tables found  
for index,table in enumerate(tables):
    table_index = index
    print(table_index)

0
1
2


In [1043]:
# Check for name of tables
for index, table in enumerate(tables):
    print(tables[index].caption)

None
None
None


In [1044]:
# Inspecting webpage and finding that table in interest is wikitable, 
# testing find all for that class, rather than looking through tables above
wikitables = soup.find_all("table",{"class":"wikitable"})
# number of tables
len(wikitables)

1

In [1045]:
# Table in interest
tablen = wikitables[0]

In [1046]:
# Create dataframe
postal_data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

# Enumerating over tablerows and columns, fill dataframe with data from table
for row in tablen.tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        PostalCode = col[0].text
        Borough = col[1].text
        Neighborhood = col[2].text.strip()
        postal_data = postal_data.append({"PostalCode":PostalCode, "Borough":Borough, "Neighborhood":Neighborhood}, ignore_index=True)

postal_data.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


### Process dataframe according to instructions

In [1047]:
# Remove(replace with empty string) newline characters in dataframe
postal_data = postal_data.replace('\n', '', regex=True)

postal_data.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [1048]:
# Create varible with True if condition met
selection = postal_data["Borough"] != "Not assigned"

# Selection from table using varieble
postal_data = postal_data[selection]

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
postal_data.loc[postal_data.Neighborhood == "Not assigned", 'Neighborhood'] = postal_data["Borough"]

# Reset index
postal_data.reset_index(drop=True, inplace=True)

# Diplay full dataframe
postal_data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [1049]:
# Number of rows and columns in dataframe
postal_data.shape

(103, 3)

## Part 2

### Import and install libraries

In [1050]:
#!pip install pgeocode # uncomment if not already installed
import pgeocode

print('Libraries imported.')

Libraries imported.


#### Find latitudes and longitudes for postcodes

In [1051]:
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = postal_data['PostalCode'].tolist()
latitudes = []
longitudes = []
for i, postal_code in enumerate(postal_codes):

    print(f'--Getting Postal Code: {postal_code}')
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        print(f'Postal Code {postal_code} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)

--Getting Postal Code: M3A
Postal Code M3A has been retrieved. 102 codes left
--Getting Postal Code: M4A
Postal Code M4A has been retrieved. 101 codes left
--Getting Postal Code: M5A
Postal Code M5A has been retrieved. 100 codes left
--Getting Postal Code: M6A
Postal Code M6A has been retrieved. 99 codes left
--Getting Postal Code: M7A
Postal Code M7A has been retrieved. 98 codes left
--Getting Postal Code: M9A
Postal Code M9A has been retrieved. 97 codes left
--Getting Postal Code: M1B
Postal Code M1B has been retrieved. 96 codes left
--Getting Postal Code: M3B
Postal Code M3B has been retrieved. 95 codes left
--Getting Postal Code: M4B
Postal Code M4B has been retrieved. 94 codes left
--Getting Postal Code: M5B
Postal Code M5B has been retrieved. 93 codes left
--Getting Postal Code: M6B
Postal Code M6B has been retrieved. 92 codes left
--Getting Postal Code: M9B
Postal Code M9B has been retrieved. 91 codes left
--Getting Postal Code: M1C
Postal Code M1C has been retrieved. 90 codes l

In [1052]:
# Add columns to dataframe
postal_data['Latitude'] = latitudes
postal_data['Longitude'] = longitudes

In [1053]:
# Display dataframe (M7R looks to be retrieved succesfully "isnot empty" but no coordinates returned)

# Replace NaN values with manually aquired latitude and longitude, for missing row
postal_data.loc[postal_data.Neighborhood == "Canada Post Gateway Processing Centre", 'Latitude'] = 43.5968
postal_data.loc[postal_data.Neighborhood == "Canada Post Gateway Processing Centre", 'Longitude'] = -79.6240

postal_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,-79.5282
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
7,M3B,North York,Don Mills,43.7450,-79.3590
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


In [1054]:
postal_data.shape

(103, 5)

## Part 3

### Import and install libraries

In [1055]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Change dataframe into more representative name

In [1056]:
toronto_data = postal_data 

#### Check number of unique boroughs

In [1058]:
print('The dataframe has {} boroughs.'.format(
        len(toronto_data['Borough'].unique())
    )
)

The dataframe has 11 boroughs.


#### Use geopy library to get the latitude and longitude values of Toronto

In [1059]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create map of Toronto using latitude and longitude values

In [1060]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [1061]:
CLIENT_ID = 'XNNAOUIZZQ2OYLXRNHFJ1Y2Z43H54LAZMPA120D1CIEXSWBJ' # your Foursquare ID
CLIENT_SECRET = 'TZA5QOUXQ1JTH31IXRNGSP1DMUFUAI1C053RS3V50OTDBVF2' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XNNAOUIZZQ2OYLXRNHFJ1Y2Z43H54LAZMPA120D1CIEXSWBJ
CLIENT_SECRET:TZA5QOUXQ1JTH31IXRNGSP1DMUFUAI1C053RS3V50OTDBVF2


#### Function to get nearby venues

In [1062]:
# Radius changed to larger, since many neighborhoods only had a few hits.

def getNearbyVenues(postal_codes, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for postal_code, lat, lng in zip(postal_codes, latitudes, longitudes):
        print(postal_code)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postal_code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Calling function for all postal codes and create new dataframe

In [1063]:
toronto_venues = getNearbyVenues(postal_codes=toronto_data['PostalCode'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


#### Print shape of dataframe and display first 5 rows

In [1064]:
print(toronto_venues.shape)
toronto_venues.head()

(4970, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.7545,-79.33,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,M3A,43.7545,-79.33,Tim Hortons,43.760668,-79.326368,Café
2,M3A,43.7545,-79.33,Brookbanks Park,43.751976,-79.332140,Park
3,M3A,43.7545,-79.33,A&W,43.760643,-79.326865,Fast Food Restaurant
4,M3A,43.7545,-79.33,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy


#### Number of venues returned for each postal code

In [1065]:
toronto_venues.groupby('PostalCode').count()

,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,13,13,13,13,13,13
M1C,4,4,4,4,4,4
M1E,39,39,39,39,39,39
M1G,10,10,10,10,10,10
M1H,18,18,18,18,18,18
M1J,10,10,10,10,10,10
M1K,19,19,19,19,19,19
M1L,23,23,23,23,23,23
M1M,14,14,14,14,14,14


#### Number of unique categories

In [1066]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 349 uniques categories.


#### One hot encoding

In [1067]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Non-Profit,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Pla

In [1068]:
toronto_onehot.shape

(4970, 350)

#### Group rows by postal code and by taking the mean of the frequency of occurrence of each category

In [1069]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.head()

,PostalCode,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Non-Profit,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Pla

In [1070]:
toronto_grouped.shape

(102, 350)

#### print each postal code along with the top 5 most common venues

In [1071]:
num_top_venues = 5

for hood in toronto_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                  venue  freq
0           Zoo Exhibit  0.69
1                  Park  0.08
2  Fast Food Restaurant  0.08
3                 Trail  0.08
4  Other Great Outdoors  0.08


----M1C----
                  venue  freq
0          Home Service  0.25
1                  Park  0.25
2  Gym / Fitness Center  0.25
3           Golf Course  0.25
4                   ATM  0.00


----M1E----
                  venue  freq
0           Pizza Place  0.10
1  Fast Food Restaurant  0.05
2        Breakfast Spot  0.05
3      Greek Restaurant  0.05
4                  Bank  0.05


----M1G----
                venue  freq
0                Park   0.2
1         Coffee Shop   0.2
2  Chinese Restaurant   0.1
3   Mobile Phone Shop   0.1
4         Pizza Place   0.1


----M1H----
                  venue  freq
0     Indian Restaurant  0.22
1           Coffee Shop  0.11
2          Burger Joint  0.06
3  Caribbean Restaurant  0.06
4                  Bank  0.06


----M1J----
               venue  freq
0  

#### Function to sort the venues in descending order

In [1072]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each postal code

In [1073]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Zoo Exhibit,Park,Fast Food Restaurant,Trail,Other Great Outdoors,Platform,Music Venue,Noodle House,Non-Profit,Nightclub
1,M1C,Home Service,Park,Gym / Fitness Center,Golf Course,ATM,New American Restaurant,Optical Shop,Office,Nudist Beach,Noodle House
2,M1E,Pizza Place,Fast Food Restaurant,Breakfast Spot,Greek Restaurant,Bank,Restaurant,Coffee Shop,Park,Liquor Store,Medical Center
3,M1G,Park,Coffee Shop,Chinese Restaurant,Mobile Phone Shop,Pizza Place,Fast Food Restaurant,Juice Bar,Pharmacy,ATM,Office
4,M1H,Indian Restaurant,Coffee Shop,Burger Joint,Caribbean Restaurant,Bank,Chinese Restaurant,Athletics & Sports,Hakka Restaurant,Gas Station,Bakery
5,M1J,Ice Cream Shop,Sandwich Place,Indian Restaurant,Big Box Store,Park,Fast Food Restaurant,Coffee Shop,Restaurant,ATM,Newsagent
6,M1K,Coffee Shop,Convenience Store,Pizza Place,Discount Store,Fast Food Restaurant,Burger Joint,Bus Line,Bank,Sandwich Place,Chinese Restaurant
7,M1L,Bakery,Convenience Store,Intersection,Coffee Shop,Vegetarian / Vegan Restaurant,Grocery Store,Sandwich Place,Park,Fast Food Restaurant,Bus Station
8,M1M,Ice Cream Shop,Sandwich Place,Park,Trail,Liquor Store,Restaurant,Pharmacy,Coffee Shop,Discount Store,Gas Station
9,M1N,Park,Ice Cream Shop,Farm,Café,General Entertainment,Gym,Gym Pool,Diner,Thai Restaurant,Skating Rink


#### kMeans clustering of postal code into 7 clusters, depending on most common venues

In [1074]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([5, 6, 1, 0, 1, 1, 0, 1, 0, 3, 1, 1, 1, 1, 1, 1, 0, 4, 6, 2, 4, 4,
       4, 0, 1, 0, 4, 0, 4, 4, 0, 0, 1, 0, 1, 1, 4, 4, 1, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 1, 1, 1, 4, 4, 4, 4, 0, 0, 0, 4, 4, 4, 4, 0, 4, 1,
       1, 4, 0, 1, 1, 3, 1, 4, 4, 1, 0, 0, 1, 1], dtype=int32)

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each postal code

In [1075]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_data = toronto_data.drop(['PostalCode'], axis=1)
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')

#  drop rows??
toronto_merged = toronto_merged.dropna()

toronto_merged = toronto_merged.reset_index(drop=True)

# Cluster as float, cast to integet, since as float not compatible with creation of map
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,1,Bus Stop,ATM,Discount Store,Fast Food Restaurant,Supermarket,Shopping Mall,Café,Tennis Court,Caribbean Restaurant,Food & Drink Shop
1,M4A,North York,Victoria Village,43.7276,-79.3148,0,Coffee Shop,Portuguese Restaurant,Pizza Place,Middle Eastern Restaurant,Park,Intersection,Thrift / Vintage Store,Sporting Goods Shop,Hockey Arena,Music Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,4,Coffee Shop,Café,Park,Theater,Restaurant,Breakfast Spot,Bakery,Gastropub,Diner,Italian Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,4,Clothing Store,Coffee Shop,Restaurant,Food Court,Fast Food Restaurant,Men's Store,Discount Store,Toy / Game Store,Fried Chicken Joint,Furniture / Home Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,4,Coffee Shop,Park,Sushi Restaurant,Café,Boutique,Hotel,Italian Restaurant,Restaurant,Clothing Store,Bookstore


#### Visualize clusters on map. postal code are clustered into 7 clusters, depending on the most common venues within radius = 1000. (Default radius = 500 did not provide enough venues for reliable clustering, for some postal code)

In [1077]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1.0).add_to(map_clusters)
       
map_clusters